# Section 1-2 - Creating Dummy Variables

In previous sections, we replaced the categorical values {C, S, Q} in the column Embarked by the numerical values {1, 2, 3}. The latter, however, has a notion of ordering not present in the former (which is simply arranged in alphabetical order). To get around this problem, we shall introduce the concept of dummy variables.

## Pandas - Extracting data

In [18]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/train.csv')

## Pandas - Cleaning data

In [19]:
df = df.drop(['Name', 'Ticket', 'Cabin'], axis=1)

age_mean = df['Age'].mean()
df['Age'] = df['Age'].fillna(age_mean)

from scipy.stats import mode

mode_embarked = mode(df['Embarked'])[0][0]
df['Embarked'] = df['Embarked'].fillna(mode_embarked)

As there are only two unique values for the column Sex, we have no problems of ordering.

In [20]:
df['Gender'] = df['Sex'].map({'female': 0, 'male': 1}).astype(int)

For the column Embarked, however, replacing {C, S, Q} by {1, 2, 3} would seem to imply the ordering C < S < Q when in fact they are simply arranged alphabetically.

To avoid this problem, we create dummy variables. Essentially this involves creating new columns to represent whether the passenger embarked at C with the value 1 if true, 0 otherwise. Pandas has a built-in function to create these columns automatically.

In [21]:
pd.get_dummies(df['Embarked'], prefix='Embarked').head(10)

,Embarked_C,Embarked_Q,Embarked_S
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1
5,0,1,0
6,0,0,1
7,0,0,1
8,0,0,1
9,1,0,0


We now concatenate the columns containing the dummy variables to our main dataframe.

In [22]:
df = pd.concat([df, pd.get_dummies(df['Embarked'], prefix='Embarked')], axis=1)
df.head(10)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Gender,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,male,22.000000,1,0,7.2500,S,1,0,0,1
1,2,1,1,female,38.000000,1,0,71.2833,C,0,1,0,0
2,3,1,3,female,26.000000,0,0,7.9250,S,0,0,0,1
3,4,1,1,female,35.000000,1,0,53.1000,S,0,0,0,1
4,5,0,3,male,35.000000,0,0,8.0500,S,1,0,0,1
5,6,0,3,male,29.699118,0,0,8.4583,Q,1,0,1,0
6,7,0,1,male,54.000000,0,0,51.8625,S,1,0,0,1
7,8,0,3,male,2.000000,3,1,21.0750,S,1,0,0,1
8,9,1,3,female,27.000000,0,2,11.1333,S,0,0,0,1
9,10,1,2,female,14.000000,1,0,30.0708,C,0,1,0,0


**Exercise**

- Write the code to create dummy variables for the column Sex.

In [24]:
pd.get_dummies(df['Sex'], prefix='Sex').head(10)
df = pd.concat([df, pd.get_dummies(df['Sex'], prefix='Sex')], axis=1)

In [25]:
df = df.drop(['Sex', 'Embarked'], axis=1)

cols = df.columns.tolist()
cols = [cols[1]] + cols[0:1] + cols[2:]
df = df[cols]

We review our processed training data.

In [26]:
df.head(10)

,Survived,PassengerId,Pclass,Age,SibSp,Parch,Fare,Gender,Embarked_C,Embarked_Q,Embarked_S,Sex_female,Sex_male
0,0,1,3,22.000000,1,0,7.2500,1,0,0,1,0,1
1,1,2,1,38.000000,1,0,71.2833,0,1,0,0,1,0
2,1,3,3,26.000000,0,0,7.9250,0,0,0,1,1,0
3,1,4,1,35.000000,1,0,53.1000,0,0,0,1,1,0
4,0,5,3,35.000000,0,0,8.0500,1,0,0,1,0,1
5,0,6,3,29.699118,0,0,8.4583,1,0,1,0,0,1
6,0,7,1,54.000000,0,0,51.8625,1,0,0,1,0,1
7,0,8,3,2.000000,3,1,21.0750,1,0,0,1,0,1
8,1,9,3,27.000000,0,2,11.1333,0,0,0,1,1,0
9,1,10,2,14.000000,1,0,30.0708,0,1,0,0,1,0


In [27]:
train_data = df.values

## Scikit-learn - Training the model

In [28]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators = 100)
model = model.fit(train_data[0:,2:],train_data[0:,0])

## Scikit-learn - Making predictions

In [46]:
df_test = pd.read_csv('../data/test.csv')

df_test = df_test.drop(['Name', 'Ticket', 'Cabin'], axis=1)

df_test['Age'] = df_test['Age'].fillna(age_mean)

fare_means = df.pivot_table('Fare', index='Pclass', aggfunc='mean')
print fare_means
df_test['Fare'] = df_test[['Fare', 'Pclass']].apply(lambda x:
                            fare_means[x['Pclass']] if pd.isnull(x['Fare'])
                            else x['Fare'], axis=1)

df_test['Gender'] = df_test['Sex'].map({'female': 0, 'male': 1}).astype(int)
df_test.head(10)

Pclass
1    84.154687
2    20.662183
3    13.675550
Name: Fare, dtype: float64


/Users/mjtiwari/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:10: FutureWarning: scalar indexers for index type Int64Index should be integers and not floating point


,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Gender
0,892,3,male,34.5,0,0,7.8292,Q,1
1,893,3,female,47.0,1,0,7.0000,S,0
2,894,2,male,62.0,0,0,9.6875,Q,1
3,895,3,male,27.0,0,0,8.6625,S,1
4,896,3,female,22.0,1,1,12.2875,S,0
5,897,3,male,14.0,0,0,9.2250,S,1
6,898,3,female,30.0,0,0,7.6292,Q,0
7,899,2,male,26.0,1,1,29.0000,S,1
8,900,3,female,18.0,0,0,7.2292,C,0
9,901,3,male,21.0,2,0,24.1500,S,1


Similarly we create dummy variables for the test data.

In [47]:
df_test = pd.concat([df_test, pd.get_dummies(df_test['Embarked'], prefix='Embarked')],
                axis=1)
df_test = pd.concat([df_test, pd.get_dummies(df_test['Sex'], prefix='Sex')], axis=1)

In [48]:
df_test = df_test.drop(['Sex', 'Embarked'], axis=1)

test_data = df_test.values

output = model.predict(test_data[:,1:])

## Pandas - Preparing for submission

In [49]:
result = np.c_[test_data[:,0].astype(int), output.astype(int)]

df_result = pd.DataFrame(result[:,0:2], columns=['PassengerId', 'Survived'])
df_result.to_csv('../results/titanic_1-2.csv', index=False)